## Table of contents
#### 1 noise removal: 
punctuation(dấu câu), stop words (stopwordsVN), URLs and Special Characters, date time, mã số tên giảng viên/ phòng dạng wzjwz
#### 2 normalization: 
Elongation, accent-marks((e.g., convert all variations of "á" to a single form)), emoji, normalize Unicode text to NFC form, lowercase, num2word, handle negation(dán nhãn cho từ phủ định)
#### 3 word segmentation
#### 4 remove non-word chars
remove all character are not word or underscore
#### 5 drop unique word
it should be processed when all above step done
#### 6 handle imbalanced data

In [106]:
pip install underthesea pyvi num2words py_vncorenlp pyvi

Note: you may need to restart the kernel to use updated packages.


In [107]:
import pandas as pd
import numpy as np

import unicodedata
from underthesea import word_tokenize
from pyvi import ViTokenizer

import re
import string

## 1. Noise removal

In [108]:
# Xử lí dấu câu
def remove_punctuation(text):
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text = text.translate(translator)
    return text

In [225]:
# Stopwords
import requests

raw_url = 'https://raw.githubusercontent.com/lavibula/SentimentAnalysis-with-Vietnamese-reviews/topic/data-preparation/vietnamese-stopwords-dash_filtered.txt'
response = requests.get(raw_url)

if response.status_code == 200:
    content = response.text
else:
    print('Failed to retrieve the file from GitHub:', response.status_code)

# Stopwords
sw = content.split('\n')
def remove_stopword(text):
    text = " ".join(x for x in text.split() if x not in sw)
    return text

In [227]:
'nhỏ' in sw

False

In [110]:
# # URLs and special character

import re

def clean_text(text):
    # Remove URLs, special characters and date time, mã số tên giảng viên/ phòng dạng wzjwz, repeating chars

    # URLs
    text = re.sub(r"https?://\S+", "", text)
    
    # special characters
    text = re.sub(r"[!@#$%/^&*(]", "", text)
    
    # dates in the format dd/mm/yyyy or dd-mm-yyyy or dd.mm.yyyy or dd_mm_yy
    text = re.sub(r'\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b', '', text)
    text = re.sub(r'\b\d{1,2}[_]\d{1,2}[_]\d{2}\b', '', text)
    
    # mã số tên giảng viên/ phòng dạng wzjwz
    text = re.sub(r'\b(wzjwz\w*)\b', '', text) 
    
    return text


## 2. Normalization

In [111]:
# Remove các ký tự kéo dài: vd: đẹppppppp
def remove_elongated_chars(text):
    replacements = {
       'a' : 'àáảãạăằắẳẵặâầấẩẫậ' ,
       'e' : 'èéẻẽẹêềếểễệ' ,
       'i' : 'ìíỉĩị' ,
       'o' : 'òóỏõọôồốổỗộơờớởỡợ' ,
       'u' : 'ùúủũụưừứửữự' ,
       'y' : 'ỳýỷỹỵ' ,
       'd' : 'đ' ,
       'A' : 'ÀÁẢÃẠĂẰẮẲẴẶÂẦẤẨẪẬ' ,
       'E' : 'ÈÉẺẼẸÊỀẾỂỄỆ' ,
       'I' : 'ÌÍỈĨỊ' ,
       'O' : 'ÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢ' ,
       'U' : 'ÙÚỦŨỤƯỪỨỬỮỰ' ,
       'Y' : 'ỲÝỶỸỴ' ,
       'D' : 'Đ' 
    }
    
    for char, replacements_str in replacements.items():
        pattern = rf"({char})\1+"
        text = re.sub(pattern, ' ' , text)
    pattern = rf"(\w)\1+"
    text = re.sub(pattern, r'\1', text)
    return text


In [112]:
#Sử dụng thư viện unicodedata để chuyển đổi các ký tự Unicode 
#tương đương thành dạng chuẩn. Ví dụ: "Hoà" thành "Hòa".
def normalize_unicode(text):
    return unicodedata.normalize("NFC", text)

In [113]:
from num2words import num2words

def handle_number(text):
    # Xử lý chuyển số thành văn bản
    words = text.split()
    cleaned_words = []
    for word in words:
        try:
            # Nếu từ kết thúc bằng dấu chấm, loại bỏ dấu chấm và xử lý số
            if word.endswith( '.' ):
                num = int(word.replace( ',' , '' )[:-1])
                word = num2words(num, lang= 'vi' ) + '.' 
            else:
                # Nếu từ chứa dấu phẩy, loại bỏ dấu phẩy và xử lý số
                if ',' in word:
                    word = num2words(float(word), lang= 'vi' )
                elif '.' in word:
                    parts = word.split( '.' ) 
                    num = '' .join(parts[0:])
                    word = num2words(int(num), lang= 'vi' )
                else:
                    num = int(word)
                    word = num2words(num, lang= 'vi' )
        except ValueError:
            # Nếu không thể chuyển đổi, giữ nguyên từ
            pass
        cleaned_words.append(word)
    
    # Kết hợp các từ thành câu
    cleaned_text = ' ' .join(cleaned_words)
    
    return cleaned_text

In [114]:
#Lowercase
def lowercase(text):
    return text.lower()

In [115]:
from pyvi import ViTokenizer

def handle_negation(text):
    
    not_words = {"không", 'không hề', "chẳng", "chưa", "không phải", "chả", "mất",
                 "thiếu", "đếch", "đéo", "kém", "nỏ", "not",
                 "bớt", "không bao giờ", "chưa bao giờ"}
    # Sắp xếp các từ từ dài nhất đến ngắn nhất
    not_words = sorted(not_words, key=len, reverse=True)

    # Thay thế các từ trong text bằng 'NOT'
    pattern = r'\b(?:' + '|'.join(re.escape(word) for word in not_words) + r')\b'
    text = re.sub(pattern, 'NOT', text, flags=re.IGNORECASE)

    return text

In [116]:
# Ví dụ sử dụng
text = "Tôi không hề muốn đi chơi, chưa bao giờ!"
processed_text = handle_negation(text)
print(processed_text)

Tôi NOT muốn đi chơi, NOT!


## 3. Word segmentation

In [119]:
from pyvi import ViTokenizer
def Word_segmentation(text):
    text = ViTokenizer.tokenize(text)
    return text

In [120]:
# Ví dụ sử dụng
text = "giáo viên cần lên lớp thường xuyên hơn và dạy những kiến thức thiết thực với môn học hơn .!"
processed_text = Word_segmentation(text)
print(processed_text)

giáo_viên cần lên_lớp thường_xuyên hơn và dạy những kiến_thức thiết_thực với môn_học hơn . !


In [164]:
# Ví dụ sử dụng
text = "nội_dung môn_học NOT trọng_tâm"
processed_text = drop_unique_word(Word_segmentation(text))
print(processed_text)

1450
nội_dung môn_học  trọng_tâm


In [159]:
# Ví dụ sử dụng
text = "chưa áp dụng công nghệ thông tin và các thiết bị hỗ trợ cho việc giảng dạy ."
processed_text = drop_unique_word(Word_segmentation(text))
print(processed_text)

1450
chưa áp_dụng công_nghệ thông_tin và các thiết_bị hỗ_trợ cho việc giảng_dạy .


## 4. Remove non-word chars

In [121]:
import re

def remove_non_word_chars(text):
    # remove non-word characters
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    return cleaned_text

text = "This is a sample_text! With punctuation& and symbols. 🌷"
cleaned_text = remove_non_word_chars(text)
print(cleaned_text)

This is a sample_text With punctuation and symbols 


## 5. Drop unique word

In [179]:
import pandas as pd

def return_unique_word(df1, column_name):
    
    df = df1.copy()
    df[column_name] = df[column_name].apply(remove_punctuation)
    df[column_name] = df[column_name].apply(clean_text)
    df[column_name] = df[column_name].apply(Word_segmentation)

    all_words_series = df[column_name].str.split(expand=True).stack()

    word_counts = all_words_series.value_counts()
    df_word_counts = pd.DataFrame({'count': word_counts}).reset_index()
    df_word_counts = df_word_counts[df_word_counts['count'] == 1]
    
    return df_word_counts['index'].values


In [180]:
from pyvi import ViTokenizer

def drop_unique_word(unique_word, text):
#     print(len(unique_word))
    # Thay thế các từ trong text bằng ''
    pattern = r'\b(?:' + '|'.join(re.escape(word) for word in unique_word) + r')\b'
    text = re.sub(pattern, '', text, flags=re.IGNORECASE)

    return text

## Preprocess data

In [168]:
def process_text(text):
    
    cleaned_text = clean_text(text)
    cleaned_text = handle_number(cleaned_text)
    cleaned_text = remove_punctuation(cleaned_text)
    cleaned_text = lowercase(cleaned_text)
    
    cleaned_text = remove_elongated_chars(cleaned_text)
    cleaned_text = normalize_unicode(cleaned_text)
    cleaned_text = Word_segmentation(cleaned_text)
    cleaned_text = handle_negation(cleaned_text)

    cleaned_text = remove_non_word_chars(cleaned_text)

    #remove nốt những ký tự thừa thãi
    cleaned_text = cleaned_text.replace(u'  ', u' ')
    cleaned_text = cleaned_text.replace(u'"', u' ')
    cleaned_text = cleaned_text.replace(u'️', u'')
    
    cleaned_text = remove_stopword(cleaned_text)
    
    return cleaned_text

In [26]:
from datasets import load_dataset

dataset = load_dataset("uitnlp/vietnamese_students_feedback")

Generating train split:   0%|          | 0/11426 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1583 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3166 [00:00<?, ? examples/s]

In [173]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 3166
    })
})

In [4]:
dataset['train']

Dataset({
    features: ['sentence', 'sentiment', 'topic'],
    num_rows: 11426
})

In [252]:
df = dataset['validation'].to_pandas()
df
# sentiment: negative, neutral, positive
# topic: lecturer, training program, facility, others

,sentence,sentiment,topic
0,giáo trình chưa cụ thể .,0,1
1,giảng buồn ngủ .,0,0
2,"giáo viên vui tính , tận tâm .",2,0
3,"giảng viên nên giao bài tập nhiều hơn , chia n...",0,0
4,"giảng viên cần giảng bài chi tiết hơn , đi sâu...",0,0
...,...,...,...
1578,hướng dẫn lab mơ hồ .,0,0
1579,thầy cho chúng em những bài tập mang tính thực...,2,0
1580,thầy không dạy nhiều chủ yếu cho sinh viên tự ...,0,0
1581,em muốn đổi tên môn học vì tên môn là lập trìn...,0,1


In [155]:
df['sentence'][:10]

0                            slide giáo trình đầy đủ .
1       nhiệt tình giảng dạy , gần gũi với sinh viên .
2                 đi học đầy đủ full điểm chuyên cần .
3    chưa áp dụng công nghệ thông tin và các thiết ...
4    thầy giảng bài hay , có nhiều bài tập ví dụ ng...
5    giảng viên đảm bảo thời gian lên lớp , tích cự...
6    em sẽ nợ môn này , nhưng em sẽ học lại ở các h...
7    thời lượng học quá dài , không đảm bảo tiếp th...
8    nội dung môn học có phần thiếu trọng tâm , hầu...
9    cần nói rõ hơn bằng cách trình bày lên bảng th...
Name: sentence, dtype: object

In [170]:
df['sentence'][:10].apply(process_text)

0                              slide giáo_trình đầy_đủ
1               nhiệt_tình giảng_dạy gần_gũi sinh_viên
2                         đi học đầy_đủ ful chuyên_cần
3    NOT áp_dụng công_nghệ thông_tin thiết_bị giảng...
4                         thầy giảng bài_tập ví_dụ lớp
5    giảng_viên lên_lớp tích_cực trả_lời câu sinh_v...
6                            nợ môn học học_kỳ kế_tiếp
7                 thời_lượng học NOT tiếp_thu hiệu_quả
8    nội_dung môn_học NOT trọng_tâm hầu_như khái_qu...
9                         trình_bày bảng thay_vì slide
Name: sentence, dtype: object

In [240]:
from tqdm import tqdm
def process_df(df):   
    column_name = 'sentence_preprocessed'
    df[column_name] = df['sentence'].apply(process_text)


In [253]:
process_df(df)
df

,sentence,sentiment,topic,sentence_preprocessed
0,giáo trình chưa cụ thể .,0,1,giáo_trình NOT
1,giảng buồn ngủ .,0,0,giảng buồn_ngủ
2,"giáo viên vui tính , tận tâm .",2,0,giáo_viên vui_tính tận_tâm
3,"giảng viên nên giao bài tập nhiều hơn , chia n...",0,0,giảng_viên giao bài_tập nhiều chia bài_tập giả...
4,"giảng viên cần giảng bài chi tiết hơn , đi sâu...",0,0,giảng_viên cần giảng chi_tiết đi_sâu code chạy...
...,...,...,...,...
1578,hướng dẫn lab mơ hồ .,0,0,hướng_dẫn lab mơ_hồ
1579,thầy cho chúng em những bài tập mang tính thực...,2,0,thầy bài_tập thực_hành thực_tiễn hài_lòng
1580,thầy không dạy nhiều chủ yếu cho sinh viên tự ...,0,0,thầy NOT dạy nhiều chủ_yếu sinh_viên
1581,em muốn đổi tên môn học vì tên môn là lập trìn...,0,1,đổi môn_học môn lập_trình c fraction cplusplus...


In [254]:
# unique word
column_name = 'sentence_preprocessed'
unique_word = return_unique_word(df, column_name)
text_dropped = []
for text in tqdm(df[column_name], desc='Processing'):
    text_dropped.append(drop_unique_word(unique_word, text))

Processing: 100%|██████████| 1583/1583 [00:01<00:00, 948.53it/s]


In [255]:
df['sentence_dropped'] = text_dropped

In [256]:
df

,sentence,sentiment,topic,sentence_preprocessed,sentence_dropped
0,giáo trình chưa cụ thể .,0,1,giáo_trình NOT,giáo_trình NOT
1,giảng buồn ngủ .,0,0,giảng buồn_ngủ,giảng buồn_ngủ
2,"giáo viên vui tính , tận tâm .",2,0,giáo_viên vui_tính tận_tâm,giáo_viên vui_tính tận_tâm
3,"giảng viên nên giao bài tập nhiều hơn , chia n...",0,0,giảng_viên giao bài_tập nhiều chia bài_tập giả...,giảng_viên giao bài_tập nhiều chia bài_tập giả...
4,"giảng viên cần giảng bài chi tiết hơn , đi sâu...",0,0,giảng_viên cần giảng chi_tiết đi_sâu code chạy...,giảng_viên cần giảng chi_tiết đi_sâu code chạy...
...,...,...,...,...,...
1578,hướng dẫn lab mơ hồ .,0,0,hướng_dẫn lab mơ_hồ,hướng_dẫn lab mơ_hồ
1579,thầy cho chúng em những bài tập mang tính thực...,2,0,thầy bài_tập thực_hành thực_tiễn hài_lòng,thầy bài_tập thực_hành thực_tiễn hài_lòng
1580,thầy không dạy nhiều chủ yếu cho sinh viên tự ...,0,0,thầy NOT dạy nhiều chủ_yếu sinh_viên,thầy NOT dạy nhiều chủ_yếu sinh_viên
1581,em muốn đổi tên môn học vì tên môn là lập trìn...,0,1,đổi môn_học môn lập_trình c fraction cplusplus...,đổi môn_học môn lập_trình c fraction học


In [257]:
df[['sentence', 'sentiment', 'topic', 'sentence_dropped']].to_csv('Validation.csv')